In [125]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas 
from pandas import DataFrame  as df
import os
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from math import log
import math
from decimal import *

In [126]:
#leitura da base e definição dos campos utilizados
#baseTotalAtributos = pandas.read_csv (r"C:\Users\Gerson\Desktop\EP2_ia2019\dataset_5secondWindow%5B3%5D.csv", usecols =  ["time","android.sensor.accelerometer#mean","android.sensor.linear_acceleration#mean","android.sensor.orientation#mean","speed#mean"])
baseTotalAtributos = pandas.read_csv (r"C:\Users\Gerson\Desktop\EP2_ia2019\dataset_5secondWindow%5B3%5D.csv", usecols =  ["time","android.sensor.accelerometer#max", "speed#max" , "speed#min","android.sensor.accelerometer#std","android.sensor.accelerometer#mean","android.sensor.linear_acceleration#mean","android.sensor.game_rotation_vector#mean", "android.sensor.accelerometer#std" ,"sound#mean", "speed#mean"])

baseTotalClasse = pandas.read_csv (r"C:\Users\Gerson\Desktop\EP2_ia2019\dataset_5secondWindow%5B3%5D.csv", usecols = ["target"])

print(baseTotalAtributos.columns.values)

['time' 'android.sensor.accelerometer#mean'
 'android.sensor.accelerometer#max' 'android.sensor.accelerometer#std'
 'android.sensor.game_rotation_vector#mean'
 'android.sensor.linear_acceleration#mean' 'sound#mean' 'speed#mean'
 'speed#min' 'speed#max']


In [127]:

#normalização por padronização 
def normalizacaoLinear(baseTotalAtr):
    colunas =  list(baseTotalAtr.columns)

    baseTotalAtributosNorm = baseTotalAtr.copy()
    baseTotalAtributosNorm[colunas] = baseTotalAtr[colunas].apply(scale)
    
    return baseTotalAtributosNorm

In [128]:
#discretização dos dados
def discretizacaoBase(baseAtributos,qtdParte):
    
    baseDiscretizada = pandas.DataFrame()
    for coluna in baseAtributos.columns.values:
        baseDiscretizada[coluna] = pandas.cut(baseAtributos[coluna],bins = qtdParte,labels= range(qtdParte))  
    return baseDiscretizada

In [129]:
def Bayer(baseTreinamento , baseTeste):
    
    totalRegistros = len(baseTreinamento)
    classes, counts_elements = np.unique(baseTreinamento['target'], return_counts=True )     
    
    colunas = baseTreinamento.columns.drop('target')
    tabelaMetricas = pandas.DataFrame(columns = ['coluna','classe','dominio','qDominios', 'qClasses','probabilidade' ])

    #treinamento ( montando uma tabela de métricas)
    for coluna in colunas:
        dominios =  baseTreinamento[coluna].unique()
        
        for dominio in dominios:
            for classe in classes:
                qtdDominioClasse   = len(baseTreinamento[(baseTreinamento[coluna]==dominio) & (baseTreinamento.target == classe)])
                qtdClasse  = len(baseTreinamento[baseTreinamento['target'] == classe ])
                p = qtdDominioClasse /qtdClasse   
                
                # caso alguma probabilidade seja 0, substituir por 0.00001 para que a multiplicação não resulte em 0
                if (p == 0):
                    p = 0.00001
                tabelaMetricas = tabelaMetricas.append ({'coluna':coluna,'classe':classe ,'dominio':dominio,'qDominios':qtdDominioClasse , 'qClasses':qtdClasse , 'probabilidade':p},ignore_index=True)
    
    #armazenando quantos registros há por classe
    qtdExemplosPorClasse = {}
    for i in range (len(classes)):
        qtdExemplosPorClasse[classes[i]] = counts_elements[i]
    
    
    classificacao = {}
    somatorio = 0
    probParcialClasses = {}
    for index, exemplo in baseTeste.iterrows():
        
        for classe in classes: 
            prob = 1            
            p2 =  qtdExemplosPorClasse[classe]/totalRegistros
            
            for coluna in colunas:
                dominio = exemplo[coluna]
                p = float(tabelaMetricas [(tabelaMetricas['coluna'] == coluna) & (tabelaMetricas['dominio'] == dominio) & (tabelaMetricas['classe'] == classe)]['probabilidade'])
               
                prob =  prob*p
                
            probParcialClasses[classe] = prob
            somatorio += prob
    
        votacaoClasse = { }
        for c in probParcialClasses :
            votacaoClasse[c] = probParcialClasses[c]/somatorio
        classificacao[index] = max(votacaoClasse, key = votacaoClasse.get)
    
    return classificacao
    
    

In [130]:

atributosNormalizados = normalizacaoLinear(baseTotalAtributos)

#particionamento da base 70 % para o conjunto de teste e 30 % para o conjunto de treinamento
baseTreinoNorm,  baseTesteNorm , classeTreinoNorm, classeTesteNorm = train_test_split(atributosNormalizados, baseTotalClasse, test_size = 0.3 , train_size = 0.7, random_state = 0)
baseTreinoNaoNorm,  baseTesteNaoNorm , classeTreinoNaoNorm, classeTesteNaoNorm = train_test_split(baseTotalAtributos, baseTotalClasse, test_size = 0.3 , train_size = 0.7, random_state = 0)


In [146]:
#DISCRETIZACAO DOS DADOS NORMALIZADOS E CONTINUOS
baseTreinamentoDiscretizadaNormalizada = discretizacaoBase(baseTreinoNorm,6)
baseTesteDiscretizadaNormalizada = discretizacaoBase(baseTesteNorm,6)

baseTreinamentoDiscretizadaNAONormalizada = discretizacaoBase(baseTreinoNaoNorm,6)
baseTesteDiscretizadaNAONormalizada = discretizacaoBase(baseTesteNaoNorm,6)

#ANEXAR A CLASSE DAS BASES UTILIZADAS ( TESTE E TREINAMENTO)
mergeTreinamentoNormalizada = baseTreinamentoDiscretizadaNormalizada.merge(classeTreinoNorm, left_index=True, right_index=True, how='inner')
mergeTesteNormalizada = baseTesteDiscretizadaNormalizada.merge(classeTesteNorm, left_index=True, right_index=True, how='inner')

mergeTreinamentoNAONormalizada = baseTreinamentoDiscretizadaNAONormalizada.merge(baseTotalClasse, left_index=True, right_index=True, how='inner')
mergeTesteNAONormalizada = baseTesteDiscretizadaNAONormalizada.merge(baseTotalClasse, left_index=True, right_index=True, how='inner')

#INVOCAR A FUNÇAO DE CLASSIFIÇACAO
setTesteNormalizado = Bayer(mergeTreinamentoNormalizada, mergeTesteNormalizada)
setTesteNAONormalizado = Bayer(mergeTreinamentoNAONormalizada, mergeTesteNAONormalizada)

In [147]:
def acuraciaKNN(setTeste,baseTotalClasse):
    acertos = 0 
    for i  in setTeste:
        if (setTeste[i] == baseTotalClasse['target'][i] ):
            acertos += 1
    print(acertos/len(setTeste))
        

In [148]:
acuraciaKNN(setTesteNormalizado, baseTotalClasse)
acuraciaKNN(setTesteNAONormalizado, baseTotalClasse)

0.4089366515837104
0.4089366515837104


In [149]:
#verificando robustez 
def mudarClasse(baseClasseTreinamento , classesDistintas,test,train):
    
    baseTempClasseTreinamento =  baseClasseTreinamento.copy()

    base_ruido ,base_nao_ruido =train_test_split(baseTempClasseTreinamento,test_size = test,  train_size = train , random_state = 0)
   
  
    for index,classe in base_ruido.iterrows():
        classesDistintasTemp = classesDistintas
        i =  np.where(classesDistintasTemp == classe['target'])
      
        
        classesDistintasTemp = np.delete(classesDistintasTemp,i)
       
        baseTempClasseTreinamento['target'][index] =  np.random.choice(classesDistintasTemp,1)[0]
       
        #print(np.random.choice(classesDistintasTemp,1)[0], type(np.random.choice(classesDistintasTemp,1)))
  
    return  baseTempClasseTreinamento 
classDistinct = baseTotalClasse['target'].unique()

In [134]:
#5%
classe5 = mudarClasse(classeTreinoNorm, classDistinct, 0.95 ,0.05 )
mergeTreinamento5 = baseTreinamentoDiscretizadaNormalizada.merge(classe5, left_index=True, right_index=True, how='inner')
setTeste5 = Bayer(mergeTreinamento5, mergeTesteNormalizada)
acuraciaKNN (setTeste5, baseTotalClasse )

0.4751131221719457


In [135]:
#10%
classe10 = mudarClasse(classeTreinoNorm, classDistinct, 0.9 ,0.1 )
mergeTreinamento10 = baseTreinamentoDiscretizadaNormalizada.merge(classe5, left_index=True, right_index=True, how='inner')
setTeste10 = Bayer(mergeTreinamento10, mergeTesteNormalizada)
acuraciaKNN (setTeste10, baseTotalClasse )

0.4751131221719457


In [136]:
#15%
classe15 = mudarClasse(classeTreinoNorm, classDistinct, 0.85 ,0.15  )
mergeTreinamento15 = baseTreinamentoDiscretizadaNormalizada.merge(classe5, left_index=True, right_index=True, how='inner')
setTeste15 = Bayer(mergeTreinamento15, mergeTesteNormalizada)
acuraciaKNN (setTeste15, baseTotalClasse )

0.4751131221719457


In [137]:
#20%
classe20 = mudarClasse(classeTreinoNorm, classDistinct, 0.8 ,0.2  )
mergeTreinamento20 = baseTreinamentoDiscretizadaNormalizada.merge(classe5, left_index=True, right_index=True, how='inner')
setTeste20 = Bayer(mergeTreinamento20, mergeTesteNormalizada)
acuraciaKNN (setTeste20, baseTotalClasse )

0.4751131221719457


In [139]:
#25%
classe25 = mudarClasse(classeTreinoNorm, classDistinct,0.75 ,0.25  )
mergeTreinamento25 = baseTreinamentoDiscretizadaNormalizada.merge(classe5, left_index=True, right_index=True, how='inner')
setTeste25 = Bayer(mergeTreinamento25, mergeTesteNormalizada)
acuraciaKNN (setTeste25, baseTotalClasse )

0.4751131221719457


In [145]:
#90%
classe25 = mudarClasse(classeTreinoNorm, classDistinct,0.1 ,0.9  )
mergeTreinamento25 = baseTreinamentoDiscretizadaNormalizada.merge(classe5, left_index=True, right_index=True, how='inner')
setTeste25 = Bayer(mergeTreinamentoNormalizada, mergeTesteNormalizada)
acuraciaKNN (setTeste25, baseTotalClasse )

0.4089366515837104
